# CHAPTER 07 - Advanced Text Generation Techniques and Tools

## Model I/O: Loading Quantized Models with LangChain

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

load_dotenv()

True

In [5]:
chat_model = ChatOpenAI(
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)

In [6]:
chat_model.invoke(
    "Hi! My name is Maarten. What is 1 + 1?"
)

AIMessage(content='Hi Maarten! 1 + 1 equals 2.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 23, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-007f8f4f-c303-4b80-abb7-41b4dd79fdc7-0', usage_metadata={'input_tokens': 23, 'output_tokens': 14, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### A Single Link in the Chain: Prompt Template

In [4]:
from langchain import PromptTemplate

In [8]:
# Create a prompt template with the "input_prompt" variable
template = """
Question: {input_prompt}
Answer:
"""

prompt_template = PromptTemplate(
    input_variables=["input_prompt"],
    template=template
)


In [9]:
basic_chain = prompt_template | chat_model

In [10]:
output = basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?"
    }
)

output.content

'Hi Maarten! The answer to 1 + 1 is 2.'

In [11]:
# Create a Chain that creates our business' name
template = """
Create a funny name for a business that sells {product}
"""

name_prompt = PromptTemplate(
    template=template,
    input_variables=["product"]
)

product_chain = name_prompt | chat_model

In [12]:
output = product_chain.invoke(
    {"product": "cars"}
)

output.content

'"The Wheely Silly Car Co."'

### A Chain with Multiple Prompts

In [13]:
from langchain import LLMChain

In [14]:
# Create a chain for the title of our story
template_title = """
Create a title for a story about {summary}. Only return the title.
Title:
"""

title_prompt = PromptTemplate(
    template=template_title,
    input_variables=["summary"]
)

title_chain = LLMChain(llm=chat_model, prompt=title_prompt, output_key="title")

C:\Users\francisco.procopio\AppData\Local\Temp\ipykernel_5684\3282791426.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title_chain = LLMChain(llm=chat_model, prompt=title_prompt, output_key="title")


In [15]:
title = title_chain.invoke(
    {
        "summary": "a girl that lost her mother"
    }
)
print(title)

{'summary': 'a girl that lost her mother', 'title': '"Eclipse of the Heart"'}


In [16]:
# Create a chain for the character description using the summary and title
template_character = """
Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.
"""

character_prompt = PromptTemplate(
    template=template_character,
    input_variables=["summary", "title"]
)

character_chain = LLMChain(
    llm = chat_model,
    prompt = character_prompt,
    output_key="character"
)

In [17]:
character = character_chain.invoke(
    {
        "summary": "a girl that lost her mother",
        "title": title['title']
    }
)

character

{'summary': 'a girl that lost her mother',
 'title': '"Eclipse of the Heart"',
 'character': 'The main character, Eliza, is a young girl who is struggling to cope with the loss of her mother and feels lost and heartbroken without her. She must navigate through her grief and find a way to heal as she searches for the light in the darkness of her life.'}

In [18]:
# Create a chain for the story using the summary, title, and character description
template_story = """
Create a story about {summary} with the title {title}. The main character is: {character}.
Only return the story and it cannot be longer than one paragraph.
"""

story_prompt = PromptTemplate(
    template=template_story,
    input_variables=["summary", "title", "character"]
)

story_chain = LLMChain(
    llm = chat_model,
    prompt = story_prompt,
    output_key="story"
)

In [19]:
story = story_chain.invoke(
    {
        "summary": "a girl that lost her mother",
        "title": title['title'],
        "character": character['character']
    }
)

story

{'summary': 'a girl that lost her mother',
 'title': '"Eclipse of the Heart"',
 'character': 'The main character, Eliza, is a young girl who is struggling to cope with the loss of her mother and feels lost and heartbroken without her. She must navigate through her grief and find a way to heal as she searches for the light in the darkness of her life.',
 'story': 'Eliza was just a young girl when she experienced the eclipse of her heart - the moment her mother passed away. From that day on, she felt like the sun had disappeared from her life, leaving her in a never-ending darkness. Eliza struggled to find her way through the confusing maze of grief and heartache, but deep down, she knew that she had to keep searching for the light that would slowly begin to shine through the shadows. As she navigated the storm of emotions within her, Eliza learned to lean on the love and support of those around her, slowly allowing her heart to heal and find its way back to the brightness that once fill

In [20]:
complete_chain = title_chain | character_chain | story_chain

In [21]:
complete_chain.invoke(
     "the best football team in the world"
    )

{'summary': 'the best football team in the world',
 'title': '"Champions of the Pitch"',
 'character': 'The main character of "Champions of the Pitch" is a skilled and determined striker who leads the team to victory with his exceptional talent and leadership on the field. They are known for their intense drive and passion for the game, setting the standard for excellence in the world of football.',
 'story': 'In the world of football, there is one team that reigns supreme - the Champions of the Pitch. Led by their skilled and determined striker, they dominate the field with exceptional talent and unrivaled leadership. With an intense drive and passion for the game, they set the standard for excellence in the world of football. Their journey to victory is marked by unforgettable moments of triumph and their legacy as the best football team in the world is etched in history forever.'}

## Memory: Helping LLMs to Remember Conversations

In [24]:
# Let's give the LLM our name
output = basic_chain.invoke({
    "input_prompt": "Hi! My name is Maarten. What is 1 + 1?"
})

resp = output.content
print(resp)

Hi Maarten! 1 + 1 equals 2.


In [26]:
# Next, we ask the LLM to reproduce the name
output = basic_chain.invoke(
    {
        "input_prompt": "What is my name?"
    }
)

resp = output.content
print(resp)

I'm sorry, I do not know your name. Can you please tell me?


### Conversation Buffer

In [29]:
from langchain.memory import ConversationBufferMemory

In [35]:
# Create an updated prompt template to include a chat history
template = """
Current conversation:\n\n{chat_history}\n\n{input_prompt}
"""

In [36]:
prompt = PromptTemplate(
    template = template,
    input_variables = ["input_prompt", "chat_history"]
)

In [37]:
# Define the type of memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=basic_chain,
    memory=memory
)

In [38]:
llm_chain.invoke({
    "input_prompt": "Hi! My name is Maarten. What is 1 + 1?"
})

{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': 'Hi Maarten! The answer to 1 + 1 is 2. How can I assist you further?'}

In [40]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({
    "input_prompt": "what is my name?"
})

{'input_prompt': 'what is my name?',
 'chat_history': 'Human: Hi! My name is Maarten. What is 1 + 1?\nAI: Hi Maarten! The answer to 1 + 1 is 2. How can I assist you further?',
 'text': 'Your name is Maarten.'}

### Windowed Conversation Buffer

In [41]:
from langchain.memory import ConversationBufferWindowMemory

In [42]:
# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(
    k=2,
    memory_key="chat_history"
)

C:\Users\francisco.procopio\AppData\Local\Temp\ipykernel_5684\3826035775.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [43]:
# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=basic_chain,
    memory=memory
)

In [44]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({
    "input_prompt": "Hi! My name is Maarten and I am 33 years old."
})

{'input_prompt': 'Hi! My name is Maarten and I am 33 years old.',
 'chat_history': '',
 'text': "Hi Maarten! Nice to meet you. What's on your mind today?"}

In [45]:
llm_chain.invoke({
    "input_prompt": "what is 3 + 3?"
})

{'input_prompt': 'what is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old.\nAI: Hi Maarten! Nice to meet you. What's on your mind today?",
 'text': '3 + 3 equals 6.'}

In [46]:
# check whether it knows the name we gave it
llm_chain.invoke({
    "input_prompt": "what is my name?"
})

{'input_prompt': 'what is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old.\nAI: Hi Maarten! Nice to meet you. What's on your mind today?\nHuman: what is 3 + 3?\nAI: 3 + 3 equals 6.",
 'text': 'Your name is Maarten.'}

In [47]:
# check whether it knows the age we gave it
llm_chain.invoke({
    "input_prompt": "what is my age?"
})

{'input_prompt': 'what is my age?',
 'chat_history': 'Human: what is 3 + 3?\nAI: 3 + 3 equals 6.\nHuman: what is my name?\nAI: Your name is Maarten.',
 'text': 'Sorry, I cannot answer that question as I do not have that information.'}

### Conversation Summary

In [51]:
from langchain.memory import ConversationSummaryMemory

In [50]:
# Create a summary prompt template
summary_prompt_template = """
Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:
"""

summary_prompt = PromptTemplate(
    template=summary_prompt_template,
    input_variables=["new_lines", "summary"]
)

print(summary_prompt.template)


Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:



In [64]:
# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=chat_model,
    memory_key="chat_history",
    prompt=summary_prompt
)

In [65]:
# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=chat_model,
    memory=memory
)

In [66]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': 'Hi Maarten! 1 + 1 equals 2. Do you have any other questions?'}

In [67]:
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Maarten asks what 1 + 1 is, and the AI responds that it equals 2. The AI then asks if Maarten has any other questions.',
 'text': "I'm sorry, I am not equipped to know your name. How can I assist you with something else?"}

In [68]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': 'Maarten asks what 1 + 1 is, and the AI responds that it equals 2. The AI then asks if Maarten has any other questions. Maarten then asks for his name, and the AI apologizes for not knowing it. The AI offers its assistance with anything else Maarten needs help with.',
 'text': 'The first question you asked was "What is 1 + 1?"'}

In [69]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': "Maarten inquires about the first question he asked, and the AI confirms that it was about the sum of 1 + 1. The AI is attentive to Maarten's queries and offers help with any other questions or needs he may have."}

## Agents: Creating a System of LLMs

### ReAct in LangChain

In [2]:
openai_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)

In [5]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times.)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}

"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [7]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

In [11]:
# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()

search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run
)

# Prepare tools
tools = load_tools(['llm-math'], llm=openai_llm)
tools.append(search_tool)

In [12]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [13]:
from langchain.agents import AgentExecutor, create_react_agent

In [14]:
# Construct the ReAct agent
agent = create_react_agent(
    openai_llm,
    tools,
    prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [16]:
# What is the price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD."
    }
)



> Entering new AgentExecutor chain...
Thought: I should use a web search engine to find the current price of a MacBook Pro in USD and then use a calculator to convert it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: With power, performance, and premium style, the MacBook Pro represents the pinnacle of Apple's laptop offerings. This comprehensive guide breaks down current MacBook Pro pricing across available sizes, processors, storage capacities, and more. You'll learn how choices at checkout impact final cost, find savings opportunities, and gain insight into Apple's pricing philosophy on its ..., title: How Much Does a MacBook Pro Cost? - The Pricer, link: https://www.thepricer.org/how-much-does-a-macbook-pro-cost/, snippet: Apple's 2024 MacBook Pro 16-inch can be equipped with an M4 Pro or M4 Max chip. Retail prices start at $2,499 USD, but every configuration is on sale, with the best prices in this guide knocking hundred of dollars off the l

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD.',
 'output': 'The current price of a MacBook Pro in USD is $2,499. It would cost approximately 2124.15 EUR with an exchange rate of 0.85 EUR for 1 USD.'}